# Import Libraries

In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# import necessary packages
import pandas as pd
from copy import deepcopy as copy
import numpy as np
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
from sklearn import datasets
from sklearn import preprocessing

# BPNN preparation

In [2]:
sablayan_clustered_df = pd.read_csv('Sablayan_Data_Clustered.csv')
sablayan_clustered_df

,Unnamed: 0,id,BARANGAY,PSGC,LAND COVERAGE,RHU,POPULATION DENSITY PER HEXAGON,POIs PER HEXAGON,ROADS,STORM SURGE HAZARD LEVEL,FLOOD HAZARD LEVEL,LANDSLIDE HAZARD LEVEL,LANDCOVER NUMERICAL,HCFAI,Cluster,Unnamed: 15
0,0.0,22.0,Ligaya,PH175109009,Brush/Shrubs,0.0,0.000000,1.0,1.0,0.0,0.0,2.0,1.0,0.000000,LOW,0
1,10.0,23.0,Burgos,PH175109009,Brush/Shrubs,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,1.0,0.000000,LOW,0
2,20.0,49.0,Pag-Asa,PH175109004,NaN,0.0,154.538807,0.0,0.0,3.0,2.0,0.0,0.0,1151.197170,MID,1
3,21.0,50.0,San Agustin,PH175109004,Built Up,0.0,214.812672,0.0,0.0,3.0,2.0,0.0,0.0,1600.191852,MID,1
4,22.0,51.0,San Agustin,PH175109004,Grassland,0.0,302.902705,0.0,0.0,3.0,1.0,0.0,1.0,2469.775420,HIGH,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,1060.0,1363.0,Pag-Asa,PH175109003,Brush/Shrubs,0.0,16.884621,0.0,0.0,0.0,0.0,3.0,1.0,235.553869,LOW,0
969,1061.0,1364.0,Pag-Asa,PH175109003,NaN,0.0,21.536779,0.0,0.0,0.0,0.0,3.0,0.0,293.994123,LOW,0
970,1062.0,1397.0,Pag-Asa,PH175109003,NaN,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.000000,LOW,0
971,1072.0,1398.0,Pag-Asa,PH175109003,NaN,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.000000,LOW,0


In [3]:
sablayan_clustered_df = sablayan_clustered_df.drop(sablayan_clustered_df.index[972])
sablayan_clustered_df

,Unnamed: 0,id,BARANGAY,PSGC,LAND COVERAGE,RHU,POPULATION DENSITY PER HEXAGON,POIs PER HEXAGON,ROADS,STORM SURGE HAZARD LEVEL,FLOOD HAZARD LEVEL,LANDSLIDE HAZARD LEVEL,LANDCOVER NUMERICAL,HCFAI,Cluster,Unnamed: 15
0,0.0,22.0,Ligaya,PH175109009,Brush/Shrubs,0.0,0.000000,1.0,1.0,0.0,0.0,2.0,1.0,0.000000,LOW,0
1,10.0,23.0,Burgos,PH175109009,Brush/Shrubs,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,1.0,0.000000,LOW,0
2,20.0,49.0,Pag-Asa,PH175109004,NaN,0.0,154.538807,0.0,0.0,3.0,2.0,0.0,0.0,1151.197170,MID,1
3,21.0,50.0,San Agustin,PH175109004,Built Up,0.0,214.812672,0.0,0.0,3.0,2.0,0.0,0.0,1600.191852,MID,1
4,22.0,51.0,San Agustin,PH175109004,Grassland,0.0,302.902705,0.0,0.0,3.0,1.0,0.0,1.0,2469.775420,HIGH,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,1059.0,1362.0,Pag-Asa,PH175109003,Open Forest,0.0,8.598491,0.0,0.0,0.0,0.0,3.0,0.0,117.376229,LOW,0
968,1060.0,1363.0,Pag-Asa,PH175109003,Brush/Shrubs,0.0,16.884621,0.0,0.0,0.0,0.0,3.0,1.0,235.553869,LOW,0
969,1061.0,1364.0,Pag-Asa,PH175109003,NaN,0.0,21.536779,0.0,0.0,0.0,0.0,3.0,0.0,293.994123,LOW,0
970,1062.0,1397.0,Pag-Asa,PH175109003,NaN,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.000000,LOW,0


In [4]:
edited_df = sablayan_clustered_df.drop(['id', 'BARANGAY', 'PSGC', 'LAND COVERAGE', 'Cluster', 'Unnamed: 0'], axis=1)

In [5]:
edited_df

,RHU,POPULATION DENSITY PER HEXAGON,POIs PER HEXAGON,ROADS,STORM SURGE HAZARD LEVEL,FLOOD HAZARD LEVEL,LANDSLIDE HAZARD LEVEL,LANDCOVER NUMERICAL,HCFAI,Unnamed: 15
0,0.0,0.000000,1.0,1.0,0.0,0.0,2.0,1.0,0.000000,0
1,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,1.0,0.000000,0
2,0.0,154.538807,0.0,0.0,3.0,2.0,0.0,0.0,1151.197170,1
3,0.0,214.812672,0.0,0.0,3.0,2.0,0.0,0.0,1600.191852,1
4,0.0,302.902705,0.0,0.0,3.0,1.0,0.0,1.0,2469.775420,2
...,...,...,...,...,...,...,...,...,...,...
967,0.0,8.598491,0.0,0.0,0.0,0.0,3.0,0.0,117.376229,0
968,0.0,16.884621,0.0,0.0,0.0,0.0,3.0,1.0,235.553869,0
969,0.0,21.536779,0.0,0.0,0.0,0.0,3.0,0.0,293.994123,0
970,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.000000,0


In [6]:
edited_df.rename(columns = {'Unnamed: 15':'Cluster'}, inplace = True)

In [7]:
# Removing mid values in Cluster column
# 1. Remove rows with Cluter values of 1
# 2. Rename clusters 2 to 1
edited_df.drop(edited_df[edited_df['Cluster'] == 1].index, inplace = True)
edited_df['Cluster'] = edited_df['Cluster'].replace(2, 1)
edited_df


,RHU,POPULATION DENSITY PER HEXAGON,POIs PER HEXAGON,ROADS,STORM SURGE HAZARD LEVEL,FLOOD HAZARD LEVEL,LANDSLIDE HAZARD LEVEL,LANDCOVER NUMERICAL,HCFAI,Cluster
0,0.0,0.000000,1.0,1.0,0.0,0.0,2.0,1.0,0.000000,0
1,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,1.0,0.000000,0
4,0.0,302.902705,0.0,0.0,3.0,1.0,0.0,1.0,2469.775420,1
5,0.0,1243.892810,13.0,1.0,3.0,2.0,2.0,0.0,5967.815673,1
6,0.0,0.000000,0.0,0.0,3.0,2.0,2.0,0.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...
967,0.0,8.598491,0.0,0.0,0.0,0.0,3.0,0.0,117.376229,0
968,0.0,16.884621,0.0,0.0,0.0,0.0,3.0,1.0,235.553869,0
969,0.0,21.536779,0.0,0.0,0.0,0.0,3.0,0.0,293.994123,0
970,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.000000,0


In [8]:
# searching for values within a andas dataframe
# if nan, replace with 0
df1 = edited_df[edited_df.isna().any(axis=1)]
df1

,RHU,POPULATION DENSITY PER HEXAGON,POIs PER HEXAGON,ROADS,STORM SURGE HAZARD LEVEL,FLOOD HAZARD LEVEL,LANDSLIDE HAZARD LEVEL,LANDCOVER NUMERICAL,HCFAI,Cluster
115,0.0,489.728984,0.0,1.0,3.0,0.0,NaN,0.0,NaN,0
176,0.0,NaN,0.0,0.0,3.0,0.0,0.0,0.0,NaN,0
197,0.0,NaN,0.0,0.0,3.0,0.0,3.0,0.0,NaN,0
822,0.0,NaN,0.0,0.0,0.0,0.0,3.0,0.0,NaN,0
877,0.0,NaN,0.0,0.0,0.0,0.0,3.0,0.0,NaN,0


In [9]:
edited_df = edited_df.fillna(0)
edited_df

,RHU,POPULATION DENSITY PER HEXAGON,POIs PER HEXAGON,ROADS,STORM SURGE HAZARD LEVEL,FLOOD HAZARD LEVEL,LANDSLIDE HAZARD LEVEL,LANDCOVER NUMERICAL,HCFAI,Cluster
0,0.0,0.000000,1.0,1.0,0.0,0.0,2.0,1.0,0.000000,0
1,0.0,0.000000,0.0,0.0,0.0,0.0,2.0,1.0,0.000000,0
4,0.0,302.902705,0.0,0.0,3.0,1.0,0.0,1.0,2469.775420,1
5,0.0,1243.892810,13.0,1.0,3.0,2.0,2.0,0.0,5967.815673,1
6,0.0,0.000000,0.0,0.0,3.0,2.0,2.0,0.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...
967,0.0,8.598491,0.0,0.0,0.0,0.0,3.0,0.0,117.376229,0
968,0.0,16.884621,0.0,0.0,0.0,0.0,3.0,1.0,235.553869,0
969,0.0,21.536779,0.0,0.0,0.0,0.0,3.0,0.0,293.994123,0
970,0.0,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.000000,0


In [10]:
x = edited_df
x_normalized = (x - x.min()) / (x.max() - x.min())
x_normalized

,RHU,POPULATION DENSITY PER HEXAGON,POIs PER HEXAGON,ROADS,STORM SURGE HAZARD LEVEL,FLOOD HAZARD LEVEL,LANDSLIDE HAZARD LEVEL,LANDCOVER NUMERICAL,HCFAI,Cluster
0,0.0,0.000000,0.021277,1.0,0.0,0.000000,0.666667,1.0,0.000000,0.0
1,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.666667,1.0,0.000000,0.0
4,0.0,0.171734,0.000000,0.0,1.0,0.333333,0.000000,1.0,0.164751,1.0
5,0.0,0.705237,0.276596,1.0,1.0,0.666667,0.666667,0.0,0.398096,1.0
6,0.0,0.000000,0.000000,0.0,1.0,0.666667,0.666667,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
967,0.0,0.004875,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.007830,0.0
968,0.0,0.009573,0.000000,0.0,0.0,0.000000,1.000000,1.0,0.015713,0.0
969,0.0,0.012210,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.019611,0.0
970,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0


In [11]:
# Remove column name 'A'
x_normalized.drop(['Cluster'], axis=1)
x_normalized

,RHU,POPULATION DENSITY PER HEXAGON,POIs PER HEXAGON,ROADS,STORM SURGE HAZARD LEVEL,FLOOD HAZARD LEVEL,LANDSLIDE HAZARD LEVEL,LANDCOVER NUMERICAL,HCFAI,Cluster
0,0.0,0.000000,0.021277,1.0,0.0,0.000000,0.666667,1.0,0.000000,0.0
1,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.666667,1.0,0.000000,0.0
4,0.0,0.171734,0.000000,0.0,1.0,0.333333,0.000000,1.0,0.164751,1.0
5,0.0,0.705237,0.276596,1.0,1.0,0.666667,0.666667,0.0,0.398096,1.0
6,0.0,0.000000,0.000000,0.0,1.0,0.666667,0.666667,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
967,0.0,0.004875,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.007830,0.0
968,0.0,0.009573,0.000000,0.0,0.0,0.000000,1.000000,1.0,0.015713,0.0
969,0.0,0.012210,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.019611,0.0
970,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0


In [12]:
y = edited_df['Cluster'].values
y

array([0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [13]:
num_features = len(x.columns)
print("Number of Features: {}".format(num_features))

Number of Features: 10


In [14]:
columns = []

for i in range(num_features):
    columns.append("x{}".format(i))

x_normalized.columns = columns # replacing the name of the columns
x_normalized

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,0.0,0.000000,0.021277,1.0,0.0,0.000000,0.666667,1.0,0.000000,0.0
1,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.666667,1.0,0.000000,0.0
4,0.0,0.171734,0.000000,0.0,1.0,0.333333,0.000000,1.0,0.164751,1.0
5,0.0,0.705237,0.276596,1.0,1.0,0.666667,0.666667,0.0,0.398096,1.0
6,0.0,0.000000,0.000000,0.0,1.0,0.666667,0.666667,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
967,0.0,0.004875,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.007830,0.0
968,0.0,0.009573,0.000000,0.0,0.0,0.000000,1.000000,1.0,0.015713,0.0
969,0.0,0.012210,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.019611,0.0
970,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.000000,0.0


In [15]:
# make y its own dataframe
# Getting the hot encoding for y (making it its own df)
# y = pd.get_dummies(y, prefix='y')
y

array([0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [16]:
y = pd.DataFrame(y, columns = ['Y_Cluster'])

print(y)
print(type(y))

     Y_Cluster
0            0
1            0
2            1
3            1
4            0
..         ...
851          0
852          0
853          0
854          0
855          0

[856 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>


# BPNN Using Pytorch - November 3, 2022

In [17]:
import torch
from torch.autograd import Variable

In [18]:
w = Variable(torch.Tensor([1.0]), requires_grad=True)

In [19]:
def forward(x):
    return x*w

In [20]:
def loss(x, y):
    y_pred = forward(x)
    return (y_pred - y) * (y_pred - y)

### pytorch

In [21]:
X = torch.tensor(x_normalized.values, dtype=torch.float)
y = torch.tensor(y.values, dtype=torch.float)

In [22]:
# We only take 2 classes to make a binary classification problem
X = X[:y[y < 2].size()[0]]
y = y[:y[y < 2].size()[0]]

print(X.size())
print(y.size())

torch.Size([856, 10])
torch.Size([856, 1])


In [23]:
type(X)

torch.Tensor

### Model Application

In [24]:
print("predict (before training)",4,forward(4).data[0])

for epoch in range(20):
    for x_val, y_val in zip(X, y):
        print(epoch)
        l = loss(x_val, y_val)
        l.mean().backward()
        # The backward() automatically computes and 
        # Updates the gradient for all variables who has requires_grad=True set True
        # Every time a variable is back propogated through,
        # the gradient will be accumulated instead of being replaced. 
        print("grad: ",x_val ,y_val ,w.grad.data[0])
        w.data = w.data - 0.01*w.grad.data
            
        # set it to zero manually after every batch    
        w.grad.data.zero_()

    print("progress:", epoch, l.data[0])
    print("\n")


print("predict (after training)",  4, forward(4).data[0])

predict (before training) 4 tensor(4.)
0


c:\Users\acer\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\autograd\__init__.py:145: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(


grad:  tensor([0.0000, 0.0000, 0.0213, 1.0000, 0.0000, 0.0000, 0.6667, 1.0000, 0.0000,
        0.0000]) tensor([0.]) tensor(0.4890)
0
grad:  tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6667, 1.0000, 0.0000,
        0.0000]) tensor([0.]) tensor(0.2875)
0
grad:  tensor([0.0000, 0.1717, 0.0000, 0.0000, 1.0000, 0.3333, 0.0000, 1.0000, 0.1648,
        1.0000]) tensor([1.]) tensor(-0.1053)
0
grad:  tensor([0.0000, 0.7052, 0.2766, 1.0000, 1.0000, 0.6667, 0.6667, 0.0000, 0.3981,
        1.0000]) tensor([1.]) tensor(-0.2246)
0
grad:  tensor([0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.6667, 0.6667, 0.0000, 0.0000,
        0.0000]) tensor([0.]) tensor(0.3761)
0
grad:  tensor([0.0000, 0.0470, 0.0000, 0.0000, 1.0000, 0.6667, 0.0000, 0.0000, 0.0412,
        0.0000]) tensor([0.]) tensor(0.2873)
0
grad:  tensor([0.0000, 0.0426, 0.0000, 1.0000, 1.0000, 0.6667, 0.0000, 0.0000, 0.0388,
        0.0000]) tensor([0.]) tensor(0.4841)
0
grad:  tensor([0.0000, 0.2202, 0.0000, 0.0000, 1.0000, 0.666